In [2]:
import pdb
import os
import errno
from pathlib import Path
import cv2
import json
import numpy as np
import matplotlib.pyplot as plt
import datetime
import random
import math
!pip3 install git+https://github.com/aleju/imgaug
from imgaug import augmenters as iaa

In [7]:
"""
    SuperVisely Object Extractor
    For each raw image containing multiple objects markted by Point, Bounding Box or Polygon we create images cropped to given 'size', with each object centered.
    We create directories for each class containing those cropped images and a single annotation file in COCO format.
    If only one class C exists you can create a 'counterclass' of same size as C, with images randomly cropped from raw images (it is guaranteed those images wont contain objects of C).
    You can chose to use 'augmentation' to expand the size of the dataset.
    'raw_annotation' describes how the raw data is annotated. So far we can distinguish between supervisely *('supervisely',)* and COCO *(coco, <path>)* annotation.
        Note: For COCO annotation the filepath is mandatory. Absolute filepaths must start a leading '/', relative filepaths must not.

    Directory Structure:

    root/
     ├───raw_data_directory/
     │
     └───Datasets/
            ├───<DatasetName>
            │       ├───annotation.json
           ...      ├───Class1/
                    ├───Class2/
                   ...
                    └───ClassN/
                    └───Fake/ (created if 'counterclass' is True)
"""

class SVObjectExtractor:
    """Prepare unprocessed Data"""

    def __init__(self, raw_path, target_path=None, dataset_name=None, object_size=None, resize_to=None, augmentation=False, counterclass=False):
        """
        Keyword arguments:
        raw_path        --  path to raw data
        target_path     --  path to save dataset at. If an existing dataset shall be expanded, give the location of the <Datasetname> directory.
        dataset_name    --  name of the dataset
        object_size     --  size of objects in images (single integer. only sqaures generated)
        resize_to       --  resize cropped images to resize_to (single integer. only squares generated)
        augmentation    --  if True, dataset will 5be expanded by augmentations of the cropped images (default False)
        counterclass    --  if True, a 'Fake' class with random images of size 'size' will be created

        Class attributes:
        RAW_DATA_PATH   --  path to raw data
        DATASET_PATH    --  path to dataset directory
        DATASET_NAME    --  name of the set to be created
        OBJECT_SIZE     --  size of cropped images around segmentation
        FINAL_SIZE      --  final size of cropped images
        AUGMENTATION    --  boolean whether augmentations of cropped images shall be created or not
        COUNTERCLASS    --  boolean, whether a fake class should be created or not
        CLASSES         --  list of classes found
        DATA_CREATED    --  boolen whether create_data was alrdy called or not
        DIRS_MADE       --  boolen whether directories are created or not
        DIR_EXISTS      --  boolean whether the Dataset directories already exist or not
        """

        print("Start initialisation..")
        if os.path.isdir(raw_path):
            self.RAW_DATA_PATH = raw_path
            invalid_path = False
            abort = False
        else:
            print("Error. Given path to raw data does not exist.")
            raise ValueError
            invalid_path = True
        if not invalid_path:
            self.DATASET_NAME = dataset_name
            self.DIRS_MADE = False
            self.SET_ALRDY_EXISTS = False

            # some cases to check, to get the right path
            setname_len = len(self.DATASET_NAME)
            if target_path:
                self.DATASET_PATH = target_path
            else:
                self.DATASET_PATH = os.getcwd()


            if self.DATASET_NAME in self.DATASET_PATH[-(setname_len+1):]:
                self.SET_ALRDY_EXISTS = True

            elif 'Datasets' in self.DATASET_PATH[-(len('Datasets')+1):]:
                if os.path.isdir(os.path.join(self.DATASET_PATH,self.DATASET_NAME)):
                    print("A directory for given datasetname already exists. The set will be expanded.")
                    if not self.__proceed():
                        raise ValueError
                        abort = True
                    else:
                        self.DATASET_PATH = os.path.join(self.DATASET_PATH,self.DATASET_NAME)
                        self.SET_ALRDY_EXISTS = True
                else:
                    self.DATASET_PATH = os.path.join(self.DATASET_PATH, self.DATASET_NAME)
            else:
                if os.path.isdir(os.path.join(self.DATASET_PATH, 'Datasets', self.DATASET_NAME)):
                    print("A path to given datasetname already exists. The set will be expanded.")
                    if not self.__proceed():
                        raise ValueError
                        abort = True
                    else:
                        self.SET_ALRDY_EXISTS = True
                self.DATASET_PATH = os.path.join(self.DATASET_PATH, 'Datasets', self.DATASET_NAME)

            if not abort:
                self.OBJECT_SIZE = object_size
                self.FINAL_SIZE = resize_to
                self.AUGMENTATION = augmentation
                self.COUNTERCLASS = counterclass
                self.CLASSES = self.__getClasses(raw_path)
                if self.COUNTERCLASS:
                    self.CLASSES.append('Fake')
                self.DATA_CREATED = False
                self.ANNOTATION = None
                self.TOTAL_IMAGES = None
                self.TOTAL_ANNOTATIONS = None

                print("Initialisation complete.")
                dir_ready = self.make_directories()
                if dir_ready:
                    self.create_data()



    """Creates a directory for each class in self.CLASSES
    """
    def make_directories(self):
        """Create direcotries for each class in self.CLASSES"""

        print("Create directories..")
        if self.SET_ALRDY_EXISTS:
            if self.DATA_CREATED:
                print("Everything is already done.")
                return False

        elif self.DIRS_MADE:
            print("Error. Directories were already created, yet could not be found.")
            return False

        try:
            for label in self.CLASSES:
                os.makedirs(os.path.join(self.DATASET_PATH, label))
        except OSError as e:
            if e.errno != errno.EEXIST:
                print(e)
                raise
                return False
            pass
        self.DIRS_MADE = True
        print("Directories created.")
        return True



    """Creates annotationfile and cropped images of raw image.
    """
    def create_data(self):
        """Creates annotationfile and cropped images of raw image."""
        print("Process Data...")
        # Annotation in COCO format
        # either load existing annotation file..
        if self.SET_ALRDY_EXISTS:
            print("Searching for existing annotation file..")
            try:
                with open(os.path.join(self.DATASET_PATH, 'annotation.json')) as annotation_file:
                    self.ANNOTATION = json.load(annotation_file)
                print("Loading file succeeded.")
                self.TOTAL_IMAGES = len(self.ANNOTATION['images'])*2
                self.TOTAL_ANNOTATIONS = len(self.ANNOTATION['annotations']) + 9000000000
            except Exception as e:
                print(e)
                print("We could not find an annotation file. If the directory is empty continue, otherwise it is not reccomended to proceed")
                if not self.__proceed():
                    return
                self.SET_ALRDY_EXISTS = False

        # ..or create new None
        if not self.SET_ALRDY_EXISTS:
            today = str(datetime.datetime.utcnow())
            categories = []
            for idx, label in enumerate(self.CLASSES):
                categories.append({'supercategory': 'object', 'id': idx, 'name': label})
            self.ANNOTATION = {
                'info': {'year': 2020,'version': None,'description': 'Pollenforager Detection','contributor': 'Mara Kortenkamp, Tim Feige','url': 'https://github.com/marakortenkamp/pollen-detection','date_created': today},
                'images': [],
                'annotations': [],
                'licenses': {'id': None,'name': None,'url': None,},
                'category': categories
                }
            self.TOTAL_IMAGES, self.TOTAL_ANNOTATIONS = 0, 9000000000

        # find image-annotation pairs
        # go find an image:
        for root, folders, files in os.walk(self.RAW_DATA_PATH):
            for folder in folders:
                if folder == 'img':
                    for img_root, img_folder, img_files in os.walk(os.path.join(root, folder)):
                        for img_file in img_files:
                            # go find its realted annotation file:
                            found_ann = False
                            for ann_root, ann_folder, ann_files in os.walk(os.path.join(root, 'ann')):
                                if found_ann:
                                    break
                                for ann_file in ann_files:
                                    # found a pair!
                                    if img_file in ann_file:
                                        # create dictionary for current img with all classes as keys and coords of objects in current image as values:
                                        # class_coords { 'class1' : [(x,y),..], 'class2' : [(x,y), (x,y), ..], .. }
                                        class_coords = { i : [] for i in self.CLASSES}
                                        with open(os.path.join(ann_root, ann_file)) as ann_json:
                                            ann_data = json.load(ann_json)
                                        if len(ann_data['objects']):
                                            for obj in ann_data['objects']:
                                                class_coords[obj['classTitle']].append(obj['points']['exterior'][0])
                                        # crop image around objects, get annotation information of each object
                                        img_path = os.path.join(img_root, img_file)
                                        self.__process_image(img_path, class_coords)
                                        # to prevent unnecessary looping:
                                        found_ann = True
                                        break
        # save annotation file
        with open(os.path.join(self.DATASET_PATH, 'annotation.json'), 'w') as fp:
            json.dump(self.ANNOTATION, fp)

        print("Data created.")



    #--------------------------#
    ##### Helper functions #####
    #--------------------------#

    def __getClasses(self,dir):
        """Return list of classes found in annotation file(s)

        Keyword arguments:
        dir        -- path to raw data

        Note: classnames must be stored in some.json: {.., 'objects': [{.., 'classTitle':<classname>, ..}, ..]}
        """
        classes = []
        for root, dicts, files in os.walk(dir):
            for file in files:
                if (file[-5:] == '.json'):
                    with open(os.path.join(root, file)) as json_file:
                        data = json.load(json_file)
                        try:
                            if len(data['objects']):
                                for object in data['objects']:
                                    if object['classTitle'] == 'est':
                                        print("est location: ", root, file)
                                    if not object['classTitle'] in classes:
                                        classes.append(object['classTitle'])
                        except:
                            pass
        return classes



    def __proceed(self):
        while True:
            proceed = input("Do you want to continue?(Y/N): ")
            if proceed.upper() == 'Y':
                return True
            elif proceed.upper() == 'N':
                return False



    def __process_image(self, filepath, coord_dic):
        """Creates cropped images for each coord in raw image and updates the COCO file.
        If requested augmented duplicates of those images are made, annotations will be provided.
        If requested, an image around random coords without annotation is created for a fake class.
        """

        img = cv2.imread(filepath, 0)
        if self.COUNTERCLASS:
            coord_dic = self.__add_rnd_coords(coord_dic)

        for label in self.CLASSES:
            for coord in coord_dic[label]:
                self.__extract_object(img, coord, label)

        return



    def __add_rnd_coords(self, coord_dic):
        """Generates a set of random coordinates. We assume that each class in self.CLASSES is about the same size,
        thus we create as many random coords as self.CLASSES[0] has in the current 'coord_dic'.
        Images cropped around random coordinates will not overlap with labeled objects.

        Note: This is optimised for the BeeProject, as it wont crop in specified areas
        Valid areas: [200,200]-[2350,3800] , [2340,50]-[2640,625] , [2330,3500]-[2640,3990]

        Note 2: Superviselys points are in (x,y) format, cv2 opens files in (y,x) format, we will save coords in (x,y) for persistence
        """

        if self.CLASSES[0] != 'Fake':
            rnd_size = len(coord_dic[self.CLASSES[0]])
        else:
            rnd_size = len(coord_dic[self.CLASSES[1]])
        
        rnd_size = len(coord_dic[self.CLASSES[idx]])

        offset = self.OBJECT_SIZE // 2

        for i in range(rnd_size):
            too_close = True
            while too_close:
                # generate rndm coord
                rnd_x = random.randint(50+offset,3990-offset)
                if (rnd_x < 200) :
                    rnd_y = random.randint(2330+offset,2640-offset)
                elif (rnd_x <= 625) :
                     rnd_y = random.randint(200+offset,2640-offset)
                elif (rnd_x < 3500) :
                    rnd_y = random.randint(200+offset,2350-offset)
                elif (rnd_x <= 3800) :
                    rnd_y = random.randint(200+offset,2640-offset)
                elif (rnd_x <= 3990) :
                    rnd_y = random.randint(2330+offset,2640-offset)
                rnd_coord = [rnd_x, rnd_y]
                # interfere with any object?
                next_try = False
                for label in self.CLASSES:
                    if next_try:
                        break
                    for coord in coord_dic[label]:
                        threshold = self.OBJECT_SIZE//2
                        too_close = self.__eukl_dist(coord, rnd_coord, threshold)
                        if too_close:
                            next_try = True
                            break

                if not too_close:
                    coord_dic['Fake'].append(rnd_coord)

        return coord_dic



    def __eukl_dist(self, p, q, threshold):
        """Returns boolean whether distance is smaller than threshold"""
        d = math.sqrt(((p[0]-q[0])**2)+((p[1]-q[1])**2))
        return (d <= threshold)



    def __extract_object(self, img, coord, label):
        """Crop image around coord,
           augment cropped image,
           save image in class directorys,
           update self.ANNOTATION for none-Fake images

           Note: cv2-Images have format [y,x], our coords have format [x,y]
           Note2: cv2 Image arrays start at 0, our coord arrays start at 1
        """

        final_images = []
        if self.AUGMENTATION:
            #for augmentation we crop a larger image, so we can rotate more easily
            object_size = self.OBJECT_SIZE+(self.OBJECT_SIZE//2)
        else:
            object_size = self.OBJECT_SIZE
        raw_img_size = img.shape        # raw_img_size has format (y,x)
        even = object_size%2
        y = coord[1]-1
        x = coord[0]-1

        start_y = y - (object_size//2)
        if ( start_y < 0 ):
            start_y = 0
            border_top = (object_size//2)-y
        else:
            border_top = 0

        end_y = start_y + object_size - border_top
        if end_y > raw_img_size[0]:
            border_bottom = raw_img_size[0]-end_y
            end_y = raw_img_size[0]
        else:
            border_bottom = 0

        start_x = x - (object_size//2)
        if ( start_x < 0 ):
            start_x = 0
            border_left = (object_size//2)-x
        else:
            border_left = 0

        end_x = start_x + object_size - border_left
        if end_x > raw_img_size[1]:
            border_right = raw_img_size[1]-end_x
            end_x = raw_img_size[1]
        else:
            border_right = 0

        object = img[start_y:end_y, start_x:end_x]
        object = cv2.copyMakeBorder(object, border_top, border_bottom, border_left, border_right, cv2.BORDER_REPLICATE)

        # create augmented versions of current cropped image
        # after augmentation, crop to size OBJECT_SIZE
        # resize_to is applied in the end
        if self.AUGMENTATION:
            final_images = self.__augment_img(object)
        else:
            if self.FINAL_SIZE:
                object = cv2.resize(object, (self.FINAL_SIZE,self.FINAL_SIZE))
            final_images.append(object)

        # safe images to storage and update annotation
        for final_image in final_images:
            self.TOTAL_IMAGES += 1
            filename = str(self.TOTAL_IMAGES) + '.png'

            if label != 'Fake':
                self.TOTAL_ANNOTATIONS += 1
                timestmp = str(datetime.datetime.utcnow())
                anno_img = {'id':self.TOTAL_IMAGES,'width':self.OBJECT_SIZE,'height':self.OBJECT_SIZE,'file_name':filename,'license':None,'flickr_url':None,'coco_url':None,'date_captured':timestmp}
                anno_anno = {'id':self.TOTAL_ANNOTATIONS,'image_id':self.TOTAL_IMAGES,'category_id':label,'segmentation':[],'area':final_image.size,'bbox':[],'iscrowd':0}
                self.ANNOTATION['images'].append(anno_img)
                self.ANNOTATION['annotations'].append(anno_anno)
            savefileat = os.path.join(self.DATASET_PATH, label, filename)
            cv2.imwrite(savefileat, final_image)

        return



    # create augmented versions of current cropped image
    # after augmentation, crop to size OBJECT_SIZE
    # resize_to is applied in the end
    def __augment_img(self, input_img):
        images = []
        final_images =[]

        # apply on input_img
        fliplr = iaa.Fliplr(1)
        flipud = iaa.Flipud(1)

        # apply on input_img and each flip
        rotate45 = iaa.Affine(rotate=(45))
        rotate90 = iaa.Rot90(1)
        rotate135 = iaa.Affine(rotate=(135))
        rotate180 = iaa.Rot90(2)
        rotate225 = iaa.Affine(rotate=(-135))
        rotate270 = iaa.Rot90(3)
        rotate315 = iaa.Affine(rotate=(-45))

        # apply on input_img, each flip and each rotation
        gauss = iaa.AdditiveGaussianNoise(scale=0.15*255)
        poison = iaa.AdditivePoissonNoise(40)
        brighter = iaa.Multiply(1.5)
        darker = iaa.Multiply(0.5)
        gaussblur = iaa.GaussianBlur(sigma=(1))
        motionblur = iaa.MotionBlur(k=3)

        final_images.append(input_img)
        for img in final_images:
            img_fliplr = fliplr(image=img)
            img_flipud = flipud(image=img)
            img_fliphv = fliplr(image=img)
            img_fliphv = flipud(image=img_fliphv)
            images.append(img)
            images.append(img_fliplr)
            images.append(img_flipud)
            images.append(img_fliphv)
        final_images = []
        for img in images:
            img_rot45 = rotate45(image=img)
            img_rot90 = rotate90(image=img)
            img_rot135 = rotate135(image=img)
            img_rot180 = rotate180(image=img)
            img_rot225 = rotate225(image=img)
            img_rot270 = rotate270(image=img)
            img_rot315 = rotate315(image=img)
            final_images.append(img)
            final_images.append(img_rot45)
            final_images.append(img_rot90)
            final_images.append(img_rot135)
            final_images.append(img_rot180)
            final_images.append(img_rot225)
            final_images.append(img_rot270)
            final_images.append(img_rot315)
        images = []
        for img in final_images:
            img_gauss = gauss(image=img)
            img_poison = poison(image=img)
            img_bright = brighter(image=img)
            img_dark = darker(image=img)
            img_gblur = gaussblur(image=img)
            img_mblur = motionblur(image=img)
            images.append(img)
            images.append(img_gauss)
            images.append(img_poison)
            images.append(img_bright)
            images.append(img_dark)
            images.append(img_gblur)
            images.append(img_mblur)

        top_crop = math.ceil((self.OBJECT_SIZE//2)/2)
        right_crop = (self.OBJECT_SIZE//2)//2
        bottom_crop = (self.OBJECT_SIZE//2)//2
        left_crop = math.ceil((self.OBJECT_SIZE//2)/2)
        img_size = input_img.shape

        final_images = []
        for img in images:
            img = img[top_crop:(img_size[0]-bottom_crop), left_crop:(img_size[1]-right_crop)]
            final_images.append(img)

        if self.FINAL_SIZE:
            for img in final_images:
                img = cv2.resize(img, (self.FINAL_SIZE,self.FINAL_SIZE))

        return final_images

In [9]:
BeeData = SVObjectExtractor(raw_path="/home/tkf/Desktop/Uni/SWP/pollen-detection/raw_data", dataset_name="PollenData", object_size=32, augmentation=True, counterclass=True)

Creating directories..
Directories created.
Creating Data..
Data created.
